In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction: Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_01_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Latent Variables & Autoencoders: Understanding the Foundation of VAEs

*Part 1 of the Vizuara series on Variational Autoencoders From Scratch*
*Estimated time: 35 minutes*

## 1. Why Does This Matter?

Imagine you want to build a machine that can *draw new faces it has never seen before*. Not copy-paste existing faces — actually **generate** brand-new, realistic faces from scratch.

How would you even begin?

The key insight is that images are not random collections of pixels. Behind every face lies a **hidden recipe** — the shape of the jaw, the color of the eyes, whether the person is smiling. If we could somehow learn these hidden recipes, we could mix and match them to create entirely new faces.

These hidden recipes are what we call **latent variables**, and learning to work with them is the foundation of some of the most powerful generative models in AI — including Variational Autoencoders (VAEs), which are the focus of this series.

In this notebook, we will:
1. Build intuition for what latent variables are using a simple thought experiment
2. Build a regular (deterministic) autoencoder on handwritten digits
3. Visualize its latent space and discover a critical flaw — the **dead zone problem**
4. Understand why this flaw motivates the probabilistic approach we will explore in Notebook 2

By the end, you will see with your own eyes *why* regular autoencoders fail at generation — and why we need something better.

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_02_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition: The Hidden Recipe

Let us start with a thought experiment. No code, no math — just intuition.

### The Cat Drawing Factory

Imagine you run a factory that produces cat drawings. Every drawing that rolls off the assembly line looks different — some cats are fat, some are skinny, some have pointy ears, some have round ears.

Now here is the crucial question: **what controls how each cat looks?**

Behind every cat drawing, there are *hidden factors* that determine its appearance:
- **Body roundness**: How chubby is the cat? (scale of 0 to 1)
- **Ear pointiness**: Are the ears sharp triangles or soft curves? (scale of 0 to 1)

These two numbers — body roundness and ear pointiness — are the **latent variables**. We call them "latent" because they are *hidden*. You never observe them directly. You only see the final drawing (the pixels), not the recipe that produced it.

Now picture a 2D plane where the x-axis is "body roundness" and the y-axis is "ear pointiness." Every possible cat drawing lives somewhere on this plane. A fat cat with pointy ears sits in the top-right corner. A skinny cat with round ears sits in the bottom-left.

This 2D plane is the **latent space** — the space of all possible hidden recipes.

Here is the magic: if we could learn a machine that takes *any point* on this 2D plane and produces a convincing cat drawing, we could generate infinite new cats just by picking random points.

That is exactly what autoencoders (and later, VAEs) try to do.

### Think About This

If you had to describe ANY cat drawing using just 2 numbers, what would those numbers represent? There is no single right answer — the point is that *some* compact description must exist, and we want our model to discover it automatically.

In [ ]:
#@title 🎧 Listen: Math Of Latent Variables
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_03_math_of_latent_variables.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics of Latent Variables

Now let us put some notation on these ideas. Do not worry — we will keep it simple.

We use two variables:

- $\mathbf{x}$ = the **observed data** (the image you can see — e.g., a 28x28 pixel image flattened into 784 numbers)
- $\mathbf{z}$ = the **latent variables** (the hidden recipe — e.g., 2 numbers that capture the "essence" of the image)

The **encoder** is a function that compresses the image into the hidden recipe:

$$\mathbf{z} = f_{\text{encoder}}(\mathbf{x})$$

In plain English: "Give me an image, and I will tell you its hidden recipe."

The **decoder** is a function that reconstructs the image from the hidden recipe:

$$\hat{\mathbf{x}} = f_{\text{decoder}}(\mathbf{z})$$

In plain English: "Give me a hidden recipe, and I will draw you the image."

Together, the encoder and decoder form an **autoencoder**. The training objective is simple: make the reconstructed image $\hat{\mathbf{x}}$ as close as possible to the original image $\mathbf{x}$.

We measure "closeness" using the **Mean Squared Error (MSE)**:

$$\mathcal{L}_{\text{recon}} = \frac{1}{N} \sum_{i=1}^{N} \| \mathbf{x}_i - \hat{\mathbf{x}}_i \|^2$$

In plain English: "For each pixel, compute the squared difference between the original and the reconstruction, then average over all pixels." The lower this number, the better our reconstruction.

### A Concrete Example

Suppose our image is just 4 pixels: $\mathbf{x} = [0.9, 0.1, 0.8, 0.2]$ and our reconstruction is $\hat{\mathbf{x}} = [0.85, 0.15, 0.75, 0.25]$.

The MSE would be:

$$\mathcal{L} = \frac{1}{4}\left[(0.9-0.85)^2 + (0.1-0.15)^2 + (0.8-0.75)^2 + (0.2-0.25)^2\right] = \frac{1}{4}(0.01) = 0.0025$$

That is a small number, which means the reconstruction is quite good.

In [ ]:
#@title 🎧 Transition: Build Autoencoder Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_04_build_autoencoder_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Build It — A Regular Autoencoder

Time to get our hands dirty. We will build a regular autoencoder and train it on MNIST handwritten digits — a dataset of 70,000 grayscale images of digits 0 through 9, each 28x28 pixels.

Why MNIST? It is small enough to train quickly, yet complex enough to reveal interesting patterns in the latent space.

### 4.1 Setup and Data

In [ ]:
#@title 🎧 Code Walkthrough: Data Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_05_data_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Install dependencies (Colab already has most of these, but just in case)
!pip install -q torch torchvision matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

print(f"Training samples: {len(train_data):,}")
print(f"Test samples:     {len(test_data):,}")
print(f"Image shape:      {train_data[0][0].shape} (channels x height x width)")
print(f"Pixel range:      [{train_data[0][0].min():.1f}, {train_data[0][0].max():.1f}]")

In [ ]:
#@title 🎧 What to Look For: Visualize Mnist
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_06_visualize_mnist.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize some samples so we know what we are working with
fig, axes = plt.subplots(2, 8, figsize=(14, 4))
for i in range(16):
    ax = axes[i // 8, i % 8]
    ax.imshow(train_data[i][0].squeeze(), cmap='gray')
    ax.set_title(str(train_data[i][1]), fontsize=10)
    ax.axis('off')
plt.suptitle('MNIST Handwritten Digits — Our Training Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Each image is 28x28 = 784 pixels.")
print("Our goal: compress these 784 numbers into just 2 latent numbers,")
print("then reconstruct the image from those 2 numbers alone.")

In [ ]:
#@title 🎧 Code Walkthrough: The Encoder
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_07_the_encoder.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 The Encoder

The encoder takes a 784-dimensional image and squeezes it through progressively smaller layers until we reach our tiny 2-dimensional latent space.

Think of it like summarizing a full novel into a single sentence — we lose details, but the *essence* should survive.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=512, latent_dim=2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)   # 784 -> 512
        self.fc2 = nn.Linear(hidden_dim, 256)          # 512 -> 256
        self.fc_out = nn.Linear(256, latent_dim)       # 256 -> 2

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc_out(h)  # A single point in latent space

# Key observation: this encoder outputs a SINGLE POINT (two numbers).
# Later, when we build a VAE, the encoder will output a DISTRIBUTION instead.
# That one change is what makes all the difference.

In [ ]:
#@title 🎧 Listen: Deterministic Mapping
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_08_deterministic_mapping.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Notice something important: this encoder maps each image to *exactly one point* in latent space. There is no randomness, no uncertainty — just a deterministic mapping. We will see why this is a problem later.

In [ ]:
#@title 🎧 Code Walkthrough: The Decoder
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_09_the_decoder.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 The Decoder

The decoder does the reverse: it takes our 2 latent numbers and expands them back to 784 pixels.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=2, hidden_dim=512, output_dim=784):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim, 256)          # 2 -> 256
        self.fc2 = nn.Linear(256, hidden_dim)          # 256 -> 512
        self.fc3 = nn.Linear(hidden_dim, output_dim)   # 512 -> 784

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return torch.sigmoid(self.fc3(h))  # Sigmoid keeps output in [0, 1]

# The sigmoid at the end ensures our pixel values stay between 0 and 1,
# matching the range of our input images.

In [ ]:
#@title 🎧 Code Walkthrough: Full Autoencoder
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_10_full_autoencoder.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 The Full Autoencoder

Now we combine the encoder and decoder into a single model. Data flows in one direction: image -> encoder -> latent code -> decoder -> reconstructed image.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=512, latent_dim=2):
        super().__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def forward(self, x):
        z = self.encoder(x)        # Compress: 784 -> 2
        x_recon = self.decoder(z)  # Reconstruct: 2 -> 784
        return x_recon, z          # Return both for visualization

In [ ]:
# Let us inspect what we have built
model = Autoencoder(latent_dim=2).to(device)
total_params = sum(p.numel() for p in model.parameters())

print(f"Total parameters: {total_params:,}")
print(f"\nArchitecture:")
print(f"  Input:   784 pixels (28x28 image)")
print(f"  Encoder: 784 -> 512 -> 256 -> 2")
print(f"  Decoder: 2 -> 256 -> 512 -> 784")
print(f"  Output:  784 pixels (reconstructed image)")
print(f"\nBottleneck: 784 dimensions compressed to just 2!")
print(f"That is a {784/2:.0f}x compression ratio.")
print(f"\nThis extreme compression forces the encoder to learn")
print(f"only the most important features of each digit.")

In [ ]:
#@title 🎧 Before You Start: Todo Training Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_11_todo_training_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn: Implement the Training Loop

Now it is time for you to write some code. The training loop for an autoencoder is straightforward:

1. **Forward pass**: Feed an image through the model, get the reconstruction
2. **Compute loss**: Measure how different the reconstruction is from the original
3. **Backward pass**: Compute gradients
4. **Update weights**: Take a step with the optimizer

We use **MSE loss with sum reduction** — this sums the squared errors across all 784 pixels for each image in the batch, then we divide by the dataset size to get the average per-image loss.

### TODO: Complete the training function

Read through the code below. The five key steps are marked with comments. The solution is filled in, but make sure you understand *why* each step is there before running it.

In [ ]:
#@title 🎧 Before You Start: Todo Training Loop Details
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_12_todo_training_loop_details.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def train_autoencoder(model, train_loader, epochs=15, lr=1e-3):
    """Train the autoencoder using MSE reconstruction loss."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    losses = []

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch_idx, (data, _) in enumerate(train_loader):
            # Flatten 28x28 images to 784-dim vectors
            data = data.view(-1, 784).to(device)

            # ============ THE FIVE STEPS ============
            # Step 1: Zero the gradients from the previous iteration
            optimizer.zero_grad()

            # Step 2: Forward pass — get reconstruction and latent code
            x_recon, z = model(data)

            # Step 3: Compute reconstruction loss (MSE, summed over pixels)
            loss = F.mse_loss(x_recon, data, reduction='sum')

            # Step 4: Backward pass — compute gradients
            loss.backward()

            # Step 5: Update the weights
            optimizer.step()
            # =========================================

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader.dataset)
        losses.append(avg_loss)
        if (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Avg Loss: {avg_loss:.4f}')

    return losses

**Exercise**: Before running the next cell, predict what will happen to the loss over time. Will it go up, go down, or stay flat? Why?

In [ ]:
#@title 🎧 Code Walkthrough: Run Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_13_run_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Train the autoencoder!
print("Training the autoencoder (this takes about 1-2 minutes)...")
print("=" * 50)
losses = train_autoencoder(model, train_loader, epochs=15)
print("=" * 50)
print("Training complete!")

In [ ]:
#@title 🎧 What to Look For: Training Curve Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_14_training_curve_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Plot the training curve
plt.figure(figsize=(8, 4))
plt.plot(losses, 'b-', linewidth=2, marker='o', markersize=4)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Average Reconstruction Loss (MSE)', fontsize=12)
plt.title('Autoencoder Training Progress', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Starting loss: {losses[0]:.4f}")
print(f"Final loss:    {losses[-1]:.4f}")
print(f"Improvement:   {(1 - losses[-1]/losses[0])*100:.1f}%")

The loss should decrease quickly at first (the model learns the broad strokes) and then plateau (the remaining details are hard to capture with only 2 latent dimensions).

In [ ]:
#@title 🎧 Transition: Reconstructions Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_15_reconstructions_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Visualizing Reconstructions

Let us see how well our autoencoder actually works. We will feed in test images it has never seen before and compare the originals to the reconstructions.

In [ ]:
#@title 🎧 What to Look For: Reconstruction Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_16_reconstruction_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Compare original vs reconstructed images
model.eval()
with torch.no_grad():
    test_batch = next(iter(test_loader))
    test_images = test_batch[0][:8].view(-1, 784).to(device)
    test_labels = test_batch[1][:8]
    recon_images, latent_codes = model(test_images)

fig, axes = plt.subplots(3, 8, figsize=(14, 6))

for i in range(8):
    # Original
    axes[0, i].imshow(test_images[i].cpu().view(28, 28), cmap='gray')
    axes[0, i].set_title(f'Digit {test_labels[i].item()}', fontsize=9)
    axes[0, i].axis('off')

    # Reconstructed
    axes[1, i].imshow(recon_images[i].cpu().view(28, 28), cmap='gray')
    axes[1, i].axis('off')

    # Difference (error map)
    diff = torch.abs(test_images[i] - recon_images[i]).cpu().view(28, 28)
    axes[2, i].imshow(diff, cmap='Reds', vmin=0, vmax=0.5)
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Reconstructed', fontsize=11, fontweight='bold')
axes[2, 0].set_ylabel('Error', fontsize=11, fontweight='bold')

plt.suptitle('Original vs Reconstructed Digits (2D Latent Space)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("The reconstructions capture the overall shape but lose fine details.")
print("This is expected — we compressed 784 dimensions into just 2!")
print(f"\nLatent codes for these images (each is just 2 numbers):")
for i in range(8):
    z = latent_codes[i].cpu().numpy()
    print(f"  Digit {test_labels[i].item()}: z = [{z[0]:.2f}, {z[1]:.2f}]")

Notice how each digit is represented by just two numbers. Different instances of the same digit tend to have similar latent codes — that is the encoder learning a meaningful representation.

In [ ]:
#@title 🎧 Transition: Dead Zone Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_17_dead_zone_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. The Dead Zone Problem

This is the most important section of the entire notebook. Everything we have built so far *works* — the autoencoder can compress and reconstruct digits. But there is a fatal flaw that prevents it from being a good **generative** model.

Let us see it with our own eyes.

### 7.1 Mapping the Latent Space

We will encode every image in the test set and plot where it lands in the 2D latent space, colored by digit class.

In [ ]:
#@title 🎧 What to Look For: Latent Space Map Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_18_latent_space_map_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Encode the entire test set and visualize the latent space
model.eval()
all_z = []
all_labels = []

with torch.no_grad():
    for data, labels in test_loader:
        data = data.view(-1, 784).to(device)
        _, z = model(data)
        all_z.append(z.cpu())
        all_labels.append(labels)

all_z = torch.cat(all_z).numpy()
all_labels = torch.cat(all_labels).numpy()

plt.figure(figsize=(10, 8))
scatter = plt.scatter(all_z[:, 0], all_z[:, 1], c=all_labels, cmap='tab10',
                       s=3, alpha=0.5)
cbar = plt.colorbar(scatter, label='Digit Class')
cbar.set_ticks(range(10))
plt.xlabel('Latent Dimension 1 ($z_1$)', fontsize=12)
plt.ylabel('Latent Dimension 2 ($z_2$)', fontsize=12)
plt.title('Regular Autoencoder Latent Space — Notice the Gaps!',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Look at this plot carefully.")
print("Each color is a different digit (0-9).")
print("Notice the EMPTY SPACES between clusters.")
print("These gaps are the 'dead zones' — regions where NO training data lives.")

Study this plot carefully. You should see distinct clusters for each digit, with **empty gaps** between them. These gaps are the crux of the problem.

In [ ]:
#@title 🎧 What to Look For: Sample Dead Zones
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_19_sample_dead_zones.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 7.2 What Happens When We Sample from Dead Zones?

If we want to use our autoencoder as a **generator**, we would pick random points in the latent space and feed them through the decoder. But what happens when we pick a point that falls in one of those empty gaps?

In [ ]:
# Sample from dead zones vs. from actual cluster regions
model.eval()

# Points from empty regions (dead zones)
# We pick coordinates that are likely in the gaps between clusters
dead_zone_points = torch.tensor([
    [0.0, 0.0],       # center of the space — likely a gap
    [-5.0, 5.0],      # far corner — probably empty
    [10.0, -10.0],    # another remote region
    [3.0, 3.0],       # midpoint between clusters
], dtype=torch.float32).to(device)

# Points from actual cluster centers (near real encodings)
# We pick indices of different digits for variety
cluster_indices = []
for digit in [0, 3, 7, 9]:
    idx = np.where(all_labels == digit)[0][0]
    cluster_indices.append(idx)
cluster_points = torch.tensor(all_z[cluster_indices], dtype=torch.float32).to(device)

with torch.no_grad():
    dead_decoded = model.decoder(dead_zone_points).cpu()
    cluster_decoded = model.decoder(cluster_points).cpu()

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
digit_names = ['0', '3', '7', '9']
dead_labels = ['(0, 0)', '(-5, 5)', '(10, -10)', '(3, 3)']

for i in range(4):
    # Top row: good samples from cluster centers
    axes[0, i].imshow(cluster_decoded[i].view(28, 28), cmap='gray')
    axes[0, i].set_title(f'Near digit {digit_names[i]}\nz = [{all_z[cluster_indices[i]][0]:.1f}, {all_z[cluster_indices[i]][1]:.1f}]',
                          fontsize=9)
    axes[0, i].axis('off')

    # Bottom row: garbage from dead zones
    axes[1, i].imshow(dead_decoded[i].view(28, 28), cmap='gray')
    axes[1, i].set_title(f'Dead zone\nz = {dead_labels[i]}', fontsize=9)
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Good regions\n(clear digits)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Dead zones\n(garbage!)', fontsize=11, fontweight='bold')
plt.suptitle('The Dead Zone Problem: Why Regular Autoencoders Fail at Generation',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("Top row: Points sampled near real data produce clean digits.")
print("Bottom row: Points sampled from gaps produce meaningless noise.")
print("\nThis is the DEAD ZONE PROBLEM.")
print("A generative model MUST produce valid outputs for ANY point in the latent space.")
print("Regular autoencoders simply do not guarantee this.")

In [ ]:
#@title 🎧 Listen: Dead Zone Explanation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_20_dead_zone_explanation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


This is the fundamental limitation. The decoder only learned to produce good outputs for the specific regions where training data exists. Everywhere else is uncharted territory — and the decoder produces garbage.

In [ ]:
#@title 🎧 What to Look For: Grid Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_21_grid_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 7.3 Visualizing Dead Zones on a Grid

Let us make this even more concrete. We will create a grid of points covering the entire latent space and decode every single one. This gives us a "map" of what the decoder produces everywhere.

In [ ]:
# Decode a grid of points across the latent space
n_grid = 20
z1_range = np.linspace(all_z[:, 0].min() - 2, all_z[:, 0].max() + 2, n_grid)
z2_range = np.linspace(all_z[:, 1].min() - 2, all_z[:, 1].max() + 2, n_grid)

# Decode each grid point
figure = np.zeros((28 * n_grid, 28 * n_grid))

model.eval()
with torch.no_grad():
    for i, z1 in enumerate(z1_range):
        for j, z2 in enumerate(z2_range):
            z = torch.tensor([[z1, z2]], dtype=torch.float32).to(device)
            decoded = model.decoder(z).cpu().view(28, 28).numpy()
            figure[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = decoded

plt.figure(figsize=(12, 12))
plt.imshow(figure, cmap='gray', origin='lower',
           extent=[z1_range[0], z1_range[-1], z2_range[0], z2_range[-1]])
plt.xlabel('Latent Dimension 1 ($z_1$)', fontsize=12)
plt.ylabel('Latent Dimension 2 ($z_2$)', fontsize=12)
plt.title('Decoder Output Across the Entire Latent Space\n(Blurry/empty regions = dead zones)',
          fontsize=14, fontweight='bold')
plt.colorbar(label='Pixel intensity')
plt.tight_layout()
plt.show()

print("This grid shows what the decoder produces for every point in latent space.")
print("Clear digits appear where training data existed.")
print("Blurry, meaningless blobs appear in the dead zones.")
print("\nA good generative model would produce clear, meaningful images EVERYWHERE.")

In [ ]:
#@title 🎧 Before You Start: Todo Latent Dim Experiment
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_22_todo_latent_dim_experiment.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Your Turn: Experiment with Latent Dimensions

We used `latent_dim=2` so we could visualize the latent space on a 2D plot. But what happens if we change the latent dimension? Does the dead zone problem get better or worse?

### TODO: Train autoencoders with different latent dimensions and compare

In [ ]:
#@title 🎧 Code Walkthrough: Run Latent Dim Experiment
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_23_run_latent_dim_experiment.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def quick_train_and_evaluate(latent_dim, epochs=10):
    """Train an autoencoder with a given latent dimension and measure quality."""
    ae = Autoencoder(latent_dim=latent_dim).to(device)
    optimizer = optim.Adam(ae.parameters(), lr=1e-3)

    # Train
    ae.train()
    for epoch in range(epochs):
        for data, _ in train_loader:
            data = data.view(-1, 784).to(device)
            optimizer.zero_grad()
            x_recon, z = ae(data)
            loss = F.mse_loss(x_recon, data, reduction='sum')
            loss.backward()
            optimizer.step()

    # Evaluate reconstruction quality on test set
    ae.eval()
    total_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.view(-1, 784).to(device)
            x_recon, _ = ae(data)
            total_loss += F.mse_loss(x_recon, data, reduction='sum').item()

    avg_loss = total_loss / len(test_data)
    return ae, avg_loss


# ============ EXPERIMENT ============
# Try different latent dimensions
dims_to_try = [2, 5, 10, 20]
results = {}

print("Training autoencoders with different latent dimensions...")
print("=" * 50)
for dim in dims_to_try:
    print(f"\nLatent dim = {dim}:")
    ae, loss = quick_train_and_evaluate(dim, epochs=10)
    results[dim] = loss
    print(f"  Test reconstruction loss: {loss:.4f}")
print("=" * 50)

In [ ]:
#@title 🎧 What to Look For: Latent Dim Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_24_latent_dim_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of reconstruction loss
dims = list(results.keys())
losses_by_dim = list(results.values())
colors = ['#e74c3c' if d == 2 else '#3498db' for d in dims]
axes[0].bar([str(d) for d in dims], losses_by_dim, color=colors, edgecolor='black')
axes[0].set_xlabel('Latent Dimension', fontsize=12)
axes[0].set_ylabel('Reconstruction Loss (lower = better)', fontsize=12)
axes[0].set_title('More Dimensions = Better Reconstruction', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Compression ratio
ratios = [784 / d for d in dims]
axes[1].bar([str(d) for d in dims], ratios, color='#2ecc71', edgecolor='black')
axes[1].set_xlabel('Latent Dimension', fontsize=12)
axes[1].set_ylabel('Compression Ratio (784 / latent_dim)', fontsize=12)
axes[1].set_title('The Compression-Quality Tradeoff', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: more latent dimensions = better reconstruction.")
print("But more dimensions also means a HIGHER-dimensional latent space,")
print("which makes the dead zone problem WORSE (exponentially more empty space).")
print("\nThis is why VAEs are so important: they solve the dead zone problem")
print("regardless of how many latent dimensions we use.")

**Exercise**: Think about this tradeoff. With 2 latent dimensions, reconstruction is poor but we can visualize the space. With 20 dimensions, reconstruction is great but the dead zones become an even bigger problem (there is exponentially more empty space in higher dimensions). How would you solve this dilemma?

In [ ]:
#@title 🎧 Code Walkthrough: Quantify Dead Zones
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_25_quantify_dead_zones.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Quantifying the Dead Zones

Let us put a number on the dead zone problem. We will estimate what fraction of the latent space is "dead" (produces garbage when decoded).

In [ ]:
def count_dead_zones(model, test_loader, device, grid_size=20):
    """Estimate how much of the latent space is 'dead' (produces garbage)."""
    model.eval()
    all_z = []
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.view(-1, 784).to(device)
            _, z = model(data)
            all_z.append(z.cpu())
    all_z = torch.cat(all_z).numpy()

    # Create a grid over the latent space (with some padding)
    z1_min, z1_max = all_z[:, 0].min() - 1, all_z[:, 0].max() + 1
    z2_min, z2_max = all_z[:, 1].min() - 1, all_z[:, 1].max() + 1
    z1_range = np.linspace(z1_min, z1_max, grid_size)
    z2_range = np.linspace(z2_min, z2_max, grid_size)

    # Count grid cells that have NO nearby encodings
    dead_count = 0
    total_count = grid_size ** 2
    for z1 in z1_range:
        for z2 in z2_range:
            distances = np.sqrt((all_z[:, 0] - z1)**2 + (all_z[:, 1] - z2)**2)
            if distances.min() > 1.0:  # No encoding within radius 1.0
                dead_count += 1

    return dead_count / total_count * 100


dead_pct = count_dead_zones(model, test_loader, device, grid_size=25)
print(f"Estimated dead zone coverage: {dead_pct:.1f}% of the latent space")
print(f"\nThis means roughly {dead_pct:.0f}% of the latent space produces garbage!")
print("If you randomly sample a point, you have about a {:.0f}% chance".format(dead_pct))
print("of getting a meaningless output.")
print("\nA VAE fixes this by forcing the encoder to spread its encodings")
print("smoothly across the latent space, filling in the gaps.")

In [ ]:
#@title 🎧 What to Look For: Interpolation Test
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_26_interpolation_test.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 10. The Motivation for VAEs: Interpolation Test

Let us do one final experiment that really drives the point home. We will **interpolate** between two digits by drawing a straight line through the latent space. In a good generative model, every point along this line should produce a valid, meaningful image.

In [ ]:
# Linear interpolation between two different digits
model.eval()
with torch.no_grad():
    # Find a '3' and an '8' in the test set
    for data, labels in test_loader:
        idx_3 = (labels == 3).nonzero(as_tuple=True)[0]
        idx_8 = (labels == 8).nonzero(as_tuple=True)[0]
        if len(idx_3) > 0 and len(idx_8) > 0:
            img_3 = data[idx_3[0]].view(-1, 784).to(device)
            img_8 = data[idx_8[0]].view(-1, 784).to(device)
            break

    z_3 = model.encoder(img_3)
    z_8 = model.encoder(img_8)

    # Interpolate between the two latent codes
    n_steps = 10
    alphas = torch.linspace(0, 1, n_steps).to(device)
    interpolated = []
    z_points = []
    for alpha in alphas:
        z_interp = (1 - alpha) * z_3 + alpha * z_8
        decoded = model.decoder(z_interp)
        interpolated.append(decoded.cpu().view(28, 28))
        z_points.append(z_interp.cpu().numpy().flatten())

# Plot the interpolation
fig, axes = plt.subplots(2, 1, figsize=(16, 5), gridspec_kw={'height_ratios': [3, 1]})

# Top: the decoded images
ax_imgs = fig.add_subplot(2, 1, 1)
combined = torch.cat([img.unsqueeze(0) for img in interpolated], dim=0)
grid_img = np.concatenate([img.numpy() for img in interpolated], axis=1)
ax_imgs.imshow(grid_img, cmap='gray')
ax_imgs.set_title('Interpolation from Digit 3 to Digit 8 in Regular Autoencoder',
                  fontsize=14, fontweight='bold')
ax_imgs.set_xticks([14 + 28*i for i in range(n_steps)])
ax_imgs.set_xticklabels([f'{a:.1f}' for a in alphas.cpu().numpy()], fontsize=9)
ax_imgs.set_xlabel('Interpolation factor (alpha)', fontsize=11)
ax_imgs.set_yticks([])

# Bottom: the latent space trajectory
z_points = np.array(z_points)
ax_traj = fig.add_subplot(2, 1, 2)
ax_traj.scatter(all_z[:, 0], all_z[:, 1], c=all_labels, cmap='tab10', s=1, alpha=0.2)
ax_traj.plot(z_points[:, 0], z_points[:, 1], 'r-o', linewidth=2, markersize=5, label='Interpolation path')
ax_traj.set_xlabel('$z_1$', fontsize=11)
ax_traj.set_ylabel('$z_2$', fontsize=11)
ax_traj.set_title('Interpolation Path Through Latent Space', fontsize=12)
ax_traj.legend(fontsize=10)
ax_traj.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice how some intermediate images look blurry or distorted?")
print("That is because the interpolation path passes through dead zones")
print("where the decoder has never seen any training data.")
print("\nIn Notebook 2, we will learn the mathematical framework (the ELBO)")
print("that fixes this by forcing the latent space to be smooth and complete.")
print("In Notebook 3, we will build a VAE and see beautiful, smooth interpolations.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_27_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 11. Key Takeaways and What is Next

Let us summarize what we have learned:

### What We Built
- An **autoencoder** with an encoder (784 -> 2) and a decoder (2 -> 784)
- The encoder compresses images into a compact latent representation
- The decoder reconstructs images from this representation

### What We Discovered
1. **Latent variables** ($\mathbf{z}$) are hidden factors that capture the essence of our data
2. A regular autoencoder maps each image to a **single point** in latent space
3. This creates **dead zones** — empty regions where the decoder produces garbage
4. The dead zone problem makes regular autoencoders **poor generators**

### Why This Matters
The dead zone problem is not just a minor inconvenience — it is a fundamental limitation. To build a true generative model, we need **every point** in the latent space to produce a valid output. This requires a fundamentally different approach: instead of mapping each image to a single point, we need to map it to a **distribution**.

### Reflection Questions
1. Why does compressing 784 dimensions to just 2 still preserve the digit identity? What does this tell us about the "true dimensionality" of handwritten digits?
2. If you increased the latent dimension from 2 to 20, would the dead zone problem get better or worse? (Hint: think about how volume scales with dimension.)
3. How is this encoder-decoder setup similar to how your brain processes visual information? When you close your eyes and imagine a "3", what is your brain's "latent code"?

### Optional Challenges
1. **Convolutional encoder**: Replace the fully-connected layers with `nn.Conv2d` and `nn.ConvTranspose2d`. Does reconstruction improve?
2. **Batch normalization**: Add `nn.BatchNorm1d` after each hidden layer. How does it affect training speed and final quality?
3. **Deeper network**: Add more hidden layers (e.g., 784 -> 512 -> 256 -> 128 -> 64 -> 2). Does it help?

### What is Next

In **Notebook 2: The ELBO and KL Divergence**, we will learn the mathematical framework that transforms our regular autoencoder into a *Variational* Autoencoder:
- The **Evidence Lower Bound (ELBO)** — the loss function that makes VAEs work
- **KL Divergence** — how we measure the "distance" between two probability distributions
- The **reparameterization trick** — the clever hack that makes it all trainable

The key idea: instead of mapping each image to a single point, the encoder will output a *probability distribution*, and the KL divergence term will ensure these distributions overlap, filling in the dead zones.

See you in Notebook 2!